In [1]:
import pandas as pd
import os
import warnings

warnings.filterwarnings('ignore')

#pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("data/turkish_movie_sentiment_dataset.csv")

print(f'shape of the dataset is: {df.shape} \n')
df.head()

shape of the dataset is: (83227, 3) 



,comment,film_name,point
0,\n Jean Reno denince zate...,Sevginin Gücü,"5,0"
1,\n Ekşın falan izlemek is...,Sevginin Gücü,"5,0"
2,\n Bu yapım hakkında öyle...,Sevginin Gücü,"5,0"
3,\n finali yeter... (sting...,Sevginin Gücü,"5,0"
4,\n Jean Reno..\r\nbu adam...,Sevginin Gücü,"5,0"


In [3]:
target = 'comment'
scores = 'point'
names  = 'film_name'

- I dont wanna remember column names everytime so I standardized with my mind like this. (Dont mind, I do this to make steps easier for me)

#### None and Duplicated Columns (check)
- Actually I'll check None values and missing inputs couple of times

In [4]:
print(df.isna().sum())

comment      0
film_name    0
point        0
dtype: int64


In [5]:
print(df.duplicated().sum())

44


In [6]:
df.drop_duplicates(inplace=True)

print(df.shape)

(83183, 3)


#### Taking movies with more than 30 comments
- I'll use a pretrained model, And for my case size of the dataset is not important.
I followed a way like this because movies with comment more than a x threshold'll be seen better in Django app. And With this method complications'll be reduces.

In [7]:
movies_w_30p_comments = df[names].value_counts()[df[names].value_counts() > 30].index

In [8]:
target_indexes = [ind for ind, name in enumerate(df[names]) if name in movies_w_30p_comments]

In [9]:
df = df.iloc[target_indexes, :].reset_index(drop=True)

In [10]:
df.head(10)

,comment,film_name,point
0,\n İlk film itibarı ile k...,Mumya,"4,0"
1,\n sırtına bolca görsel e...,Mumya,"2,0"
2,\n bence gayet güzel bi f...,Mumya,"5,0"
3,\n Sinemada seyretmiştim....,Mumya,"4,0"
4,\n harikaaa bi filmdi.. i...,Mumya,"4,5"
5,\n filmi birkaç defa izle...,Mumya,"4,5"
6,\n bu zamana kadar izledi...,Mumya,"4,0"
7,\n Ani efektleri ile bir ...,Mumya,"4,0"
8,\n çok güzel ve görsel bi...,Mumya,"4,5"
9,\n Ben büyük bir heyecanl...,Mumya,"4,0"


### Dropping comments with more than 200 words
I think long comments could be complicated and In my interface, they wont seem good

In [11]:
long_comments = [ind for ind, comment in enumerate(df[target]) if len(comment.split()) > 200]

print(f'Number of the problematic comments bc of size is: {len(long_comments)} ot of {len(df)}')

Number of the problematic comments bc of size is: 95 ot of 3437


In [12]:
df = df[~df.index.isin(long_comments)].reset_index(drop=True)

print(f'len of our dataset is: {len(df)}')

len of our dataset is: 3342


### Cleaning Phase
There is some data cleaning steps are needed. Actually NLP requires some preprocessing, but beyond the NLP preprocessing steps there are some easily seen steps.
- Begining of the comments there is '\n',
- And scores in the 'point' columns arent seen as float bc of ',' (they should be '.')

In [13]:
# Converting point column from string to float
df[scores] = df[scores].apply(lambda x: x.replace(',', '.'))
df[scores] = df[scores].astype(float)

In [14]:
# We can see some \n at the begining and the ending of the comments. And there is some more spaces actually
test_comment = df[target][0]
test_comment

"\n                      İlk film itibarı ile kadroda nam yapmış star oyuncu olmamasına rağmen iyi bir iş çıkarılmış. Efekt ve görsel sunum çok iyiydi. Brendan Fraser ve Beni karakterini oynayan oyuncu en başarılı oyunculardı. Rachel Weisz kanımca çok zayıftı. Fotojenik bir yüzü ve fiziksel açıdan hiçbir özelliği olmamasına rağmen bir Holywood yapımında oynaması ilginç. Film Fas'ta çekilmiş. O yıllara(1923) ait kostümler iyi dizayn edilmiş. 80 milyon dolar bütçeli film 450 milyon dolara yakın global bir gişe başarısı elde etmiş.\n        \n            "

In [15]:
# Actually this one didnt solve our problem directly
test_comment.replace('\n', '')

"                      İlk film itibarı ile kadroda nam yapmış star oyuncu olmamasına rağmen iyi bir iş çıkarılmış. Efekt ve görsel sunum çok iyiydi. Brendan Fraser ve Beni karakterini oynayan oyuncu en başarılı oyunculardı. Rachel Weisz kanımca çok zayıftı. Fotojenik bir yüzü ve fiziksel açıdan hiçbir özelliği olmamasına rağmen bir Holywood yapımında oynaması ilginç. Film Fas'ta çekilmiş. O yıllara(1923) ait kostümler iyi dizayn edilmiş. 80 milyon dolar bütçeli film 450 milyon dolara yakın global bir gişe başarısı elde etmiş.                    "

In [16]:
first = test_comment.find('İlk')

last = test_comment.find('etmiş.') # length of the etmis. = 6

number_of_ending_spaces = len(test_comment) - last - 6

print(f"First {first} character is 'space' and the last {number_of_ending_spaces} character is also 'space' in our case (for first comment)")

First 23 character is 'space' and the last 22 character is also 'space' in our case (for first comment)


In [17]:
# So in our case If I pass first 23 characters and last 22 characters of my comment, spaces are gonna be removed.

test_comment[23:-22]

"İlk film itibarı ile kadroda nam yapmış star oyuncu olmamasına rağmen iyi bir iş çıkarılmış. Efekt ve görsel sunum çok iyiydi. Brendan Fraser ve Beni karakterini oynayan oyuncu en başarılı oyunculardı. Rachel Weisz kanımca çok zayıftı. Fotojenik bir yüzü ve fiziksel açıdan hiçbir özelliği olmamasına rağmen bir Holywood yapımında oynaması ilginç. Film Fas'ta çekilmiş. O yıllara(1923) ait kostümler iyi dizayn edilmiş. 80 milyon dolar bütçeli film 450 milyon dolara yakın global bir gişe başarısı elde etmiş."

- Looks good. But I have to test this for other comments (I'll show some of them)

In [18]:
for comment in df[target][:5]:
    print(comment[23:-22], '\n')

İlk film itibarı ile kadroda nam yapmış star oyuncu olmamasına rağmen iyi bir iş çıkarılmış. Efekt ve görsel sunum çok iyiydi. Brendan Fraser ve Beni karakterini oynayan oyuncu en başarılı oyunculardı. Rachel Weisz kanımca çok zayıftı. Fotojenik bir yüzü ve fiziksel açıdan hiçbir özelliği olmamasına rağmen bir Holywood yapımında oynaması ilginç. Film Fas'ta çekilmiş. O yıllara(1923) ait kostümler iyi dizayn edilmiş. 80 milyon dolar bütçeli film 450 milyon dolara yakın global bir gişe başarısı elde etmiş. 

sırtına bolca görsel efetlere dayamış, gerek hikayesi gerekse karakterleri ile sempatik olamayan bir film. Temposuyla yorucu ve sıkıcı bir film.4/10 

bence gayet güzel bi filmdi her ne kadar brendan fraser kendi kulvarının dışına çıksada yinede bence mükemmel bi oyunculuk gösterisinde bulunmuş....kaç defada seyrettiğimi bilmiyom ama her seyrettiğimde ayrı güzel geldi...ama bu filmide korku kategorisine almalarına hayret ettim doğrusu, ne alakaysa.:??? 

Sinemada seyretmiştim.Gerçekt

- Yup, everything looks good.

In [19]:
df[target] = df[target].apply(lambda x: x[23:-22])

In [20]:
df['original_comments'] = df[target].copy()

### NLP Preprocessing

In [21]:
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

In [22]:
#!pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html


In [23]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

model = AutoModelForSequenceClassification.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-sentiment-cased")

sa = pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

In [24]:
import re
import sys
from nltk.stem.porter import PorterStemmer

def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    return word


def preprocess_text(text):
    processed_text = []
    
    text = text.lower()

    # Replace 2+ or more dots with space
    text = re.sub(r'\.{2,}', ' ', text)
    
    # Strip space, " and '
    text = text.strip(' "\'')

    text = re.sub(r'\s+', ' ', text)
    words = text.split()

    for word in words:
      word = preprocess_word(word)
      processed_text.append(word)

    return ' '.join(processed_text)


In [25]:
def sentiment_analysis(text):
    "True if positive"
    p = sa(str(text))
    return pd.Series([p[0]["label"], round(p[0]["score"],2)])


In [ ]:
df["sentiment_label"] = "-"
df[["sentiment_label", "sentiment_score"]] = df[target].progress_apply(sentiment_analysis)


In [34]:
df_to_save = df[[names, 'sentiment_label', 'original_comments']]

df_to_save.columns = ['movie', 'sentiment', 'comment']

df_to_save.to_csv('data/predicteds.csv', index=False)

In [35]:
df[names].nunique()

86

### Check Success Rate

In [36]:
gr25 = (df[scores] > 2.5).sum()
tp_gr25 = ((df[scores] > 2.5) & (df['sentiment_label'] == 'positive')).sum()

lw25 = (df[scores] < 2.5).sum()
tn_lw25 = ((df[scores] < 2.5) & (df['sentiment_label'] != 'positive')).sum()
print(f"""
Total numbers of point with greater than 2.5 are {gr25} and the # of True classified is {tp_gr25}, True classified rate: {tp_gr25 / gr25} \n
Total numbers of point with greater than 2.5 are {lw25} and the # of True classified is {tn_lw25}, True classified rate: {tn_lw25 / lw25}
""")


Total numbers of point with greater than 2.5 are 2264 and the # of True classified is 1814, True classified rate: 0.8012367491166078 

Total numbers of point with greater than 2.5 are 545 and the # of True classified is 507, True classified rate: 0.9302752293577982



* Success metrics is not bed, so I wont do any additional regularization.